In [1]:
RUBY_CMD = 'ruby'
BUNDLE_CMD = 'bundle exec ruby'
META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
PROJECT = File.absolute_path('/opt/OpenStudio-server/server/spec/files')
HOST = '192.168.65.3'

"192.168.65.3"

In [2]:
@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD
@bundle_cmd = BUNDLE_CMD

"bundle exec ruby"

In [3]:
#Single Run

In [4]:
command = "#{@meta_cli} run_analysis --debug --verbose '#{@project}/SEB_calibration_single_run_2013.json' 'http://#{@host}' -z 'SEB_calibration_NSGA_2013' -a single_run"

"/opt/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose '/opt/OpenStudio-server/server/spec/files/SEB_calibration_single_run_2013.json' 'http://192.168.65.3' -z 'SEB_calibration_NSGA_2013' -a single_run"

In [5]:
run_analysis = system(command)

true

In [6]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"e10afc98-763f-4496-a9e7-b87de7278b68"

In [7]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:status]

"started"

In [8]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:problem][:analysis_type]

"single_run"

In [9]:
a[:analysis][:urbanopt]

false

In [10]:
# get all data points in this analysis
a = RestClient.get "http://#{@host}/data_points.json"
a = JSON.parse(a, symbolize_names: true)
data_points = []
a.each do |data_point|
  if data_point[:analysis_id] == analysis_id
    data_points << data_point
  end
end
data_points.size

1

In [11]:
data_point_id = data_points[0][:_id]

"ec654306-de41-498f-94b9-d968f8578248"

In [14]:
dp = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
dp = JSON.parse(dp, symbolize_names: true)
data_points_status = dp[:data_point][:status]

"completed"

In [15]:
dp[:data_point][:status_message]

"completed normal"

In [16]:
results = dp[:data_point][:results][:calibration_reports_enhanced_20]

{:electric_nmbe_limit=>5.0, :electric_cvrmse_limit=>15.0, :gas_nmbe_limit=>5.0, :gas_cvrmse_limit=>15.0, :ashrae_max_nmbe=>5.0, :ashrae_max_cvrmse=>15.0, :femp_max_nmbe=>15.0, :femp_max_cvrmse=>10.0, :electricity_consumption_cvrmse=>34.85459811334266, :electricity_consumption_nmbe=>-35.591021413089216, :electricity_period_1_start_date=>"2013-Jan-01", :electricity_period_1_end_date=>"2013-Jan-30", :electricity_period_1_consumption_actual=>1562.0, :electricity_period_1_consumption_modeled=>1973.4347488620274, :electricity_period_1_peak_demand_modeled=>6.313304755609102, :electricity_period_1_consumption_nmbe=>26.340252808068335, :electricity_period_2_start_date=>"2013-Feb-01", :electricity_period_2_end_date=>"2013-Feb-28", :electricity_period_2_consumption_actual=>1395.0, :electricity_period_2_consumption_modeled=>1749.6053975455463, :electricity_period_2_peak_demand_modeled=>4.913394065262417, :electricity_period_2_consumption_nmbe=>25.419741759537374, :electricity_period_3_start_date=>

In [17]:
results.slice(:electricity_consumption_cvrmse, :electricity_consumption_nmbe, :natural_gas_consumption_cvrmse, :natural_gas_consumption_nmbe)

{:electricity_consumption_cvrmse=>34.85459811334266, :electricity_consumption_nmbe=>-35.591021413089216, :natural_gas_consumption_cvrmse=>162.94187840634328, :natural_gas_consumption_nmbe=>-130.58339588071794}

In [18]:
objectives_json = RestClient.get "http://#{@host}/data_points/#{data_point_id}/download_result_file?filename=objectives.json"
objectives_json = JSON.parse(objectives_json, symbolize_names: true)

{:objective_function_1=>34.85459811334266, :objective_function_target_1=>0.0, :objective_function_group_1=>1.0, :objective_function_2=>-35.591021413089216, :objective_function_target_2=>0.0, :objective_function_group_2=>2.0, :objective_function_3=>162.94187840634328, :objective_function_target_3=>0.0, :objective_function_group_3=>3.0, :objective_function_4=>-130.58339588071794, :objective_function_target_4=>0.0, :objective_function_group_4=>4.0}

In [19]:
require 'open-uri'
File.write 'results.csv', URI.open("http://#{@host}/analyses/#{analysis_id}/download_data.csv?export=true").read


2207